In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.metrics import classification_report
import pickle

from SistemaHierarquico import Sistema_Hierarquico_Base
from utils.system_evaluation import test_sistema


In [21]:
with open(r"Models\reprodução\FirstStage.p",'rb') as f:
    first_stage = pickle.load(f)
first_stage

Pipeline(steps=[('scaler',
                 QuantileTransformer(output_distribution='normal',
                                     subsample=100000)),
                ('pca', PCA(n_components=56, random_state=42)),
                ('ocsvm',
                 OneClassSVM(gamma=0.0632653906314333, nu=0.0002316646233151,
                             verbose=True))])

In [30]:
with open(r"Models\reprodução\SecondStage.p","rb") as f:
    second_stage = pickle.load(f)
second_stage

Pipeline(steps=[('scaler',
                 QuantileTransformer(output_distribution='normal',
                                     subsample=100000)),
                ('floresta',
                 RandomForestClassifier(max_features=0.1751204590963604,
                                        max_samples=0.9034128710297624,
                                        n_estimators=97, random_state=42))])

In [23]:
class SistemaHierarquico(Sistema_Hierarquico_Base):
        def __init__(self,anomaly_detector, classifier, n_classes, threshold_b, threshold_m, threshold_u):
            super().__init__(anomaly_detector, classifier, n_classes, threshold_b, threshold_m, threshold_u)

        def detect_anomaly_proba(self, X):
            # vetor com classe de maior probabilidade de cada instancia
            # pegam so a probabilidade dos elementos serem 1
            y_score = self.anomaly_detector.score_samples(X)

            # normaliza para ficar entre 0 e 1 como uma probabilidade
            y_score = (y_score - y_score.min())/(y_score.max() -y_score.min())

            # como valoores maiores sao de benignos, precisamos inverter
            y_score = 1-y_score
            return y_score

        def classify_proba(self,X):
            # matriz, cada linha representa uma instancia que pde ser maligna,
            # cada coluna representa uma classe
            # cada elemento da matriz representa uma probabilidade
            return self.classifier.predict_proba(X)

In [24]:
test_full_model   = pd.read_csv("data\\test_full_model.csv")
test_set          = pd.read_csv("data\\test_dataset.csv")

print("test_dataset")
print(test_set.info())
print("-"*100, "\n\n")

print("test_full_model")
print(test_full_model.info())
print("-"*100, "\n\n")



test_dataset
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56860 entries, 0 to 56859
Data columns (total 46 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              56860 non-null  int64  
 1   Protocol                56860 non-null  float64
 2   Fwd Packet Length Min   56860 non-null  float64
 3   Fwd Packet Length Std   56860 non-null  float64
 4   Bwd Packet Length Min   56860 non-null  float64
 5   Flow Bytes/s            56860 non-null  float64
 6   Flow IAT Std            56860 non-null  float64
 7   Fwd IAT Total           56860 non-null  float64
 8   Fwd IAT Mean            56860 non-null  float64
 9   Fwd IAT Std             56860 non-null  float64
 10  Fwd IAT Min             56860 non-null  float64
 11  Bwd IAT Total           56860 non-null  float64
 12  Bwd IAT Mean            56860 non-null  float64
 13  Bwd IAT Std             56860 non-null  float64
 14  Bwd IAT Max             5

In [25]:
nome_anomalias = list(second_stage.classes_)
nome_labels    = ["Unknown"] + nome_anomalias + ["BENIGN"]
nome_labels

['Unknown',
 '(D)DOS',
 'Botnet',
 'Brute Force',
 'Port Scan',
 'Web-Attack',
 'BENIGN']

In [26]:
test_dataset_list = [
                        test_full_model,
                        test_set           
                    ]
test_dataset_names = ["test_full_model","test_set"]

In [27]:
sh = SistemaHierarquico(first_stage,second_stage, 5, .25, .85, .9) 

In [28]:
test_sistema(sh,test_dataset_list, test_dataset_names, nome_labels)

ValueError: X has 55 features, but QuantileTransformer is expecting 66 features as input.

In [ ]:
time_spent_each_stage = sh.get_execution_time_lists()

print("tempos em exec (segundos)")
pd.DataFrame([np.round(time_spent,2) for  time_spent in time_spent_each_stage], columns= test_dataset_names, index= ["Stage1", "Stage2", "Stage3", "Sistema"])
    

tempos em exec (segundos)


,test_full_model,test_set
Stage1,5.36,2.84
Stage2,0.76,0.43
Stage3,0.00,0.00
Sistema,6.12,3.27
